In [13]:
import json

import numpy as np
import pandas as pd
from scipy.optimize import least_squares

In [14]:
data_path = "../../data"

In [15]:
## Reverse-engineered matching road graph nodes
# Nodes in unknown projection in the MMVC road graph
mmvc_node_ids = [105, 741, 90, 732, 567, 580, 924, 6099, 59, 601, 1351, 133]
# Nodes in WGS84 in the CSVT road graph
csvt_node_ids = [6091, 3891, 3815, 4582, 3890, 3808, 3809, 10529, 6927, 4237, 3835, 4218]

In [16]:
nodes_path = f"{data_path}/road_graph/node_shenzhen.csv"
edges_path = f"{data_path}/road_graph/edge_shenzhen.csv"

nodes_df = pd.read_csv(nodes_path)
edges_df = pd.read_csv(edges_path)

# Fetch node coordinates
csvt_nodes_dict = dict()
for _, row in nodes_df.iterrows():
    node_id = int(row["NodeID"])
    if node_id in csvt_node_ids:
        csvt_nodes_dict[node_id] = {"lon": float(row["Longitude"]), "lat": float(row["Latitude"])}

mmvc_nodes_dict = dict()
with open(f"{data_path}/dataset/map.json", mode="r", encoding="utf-8") as file:
    items = json.load(file)

    for item in items:
        node_id = item["id"]
        if item["type"] == "node" and node_id in mmvc_node_ids:
            mmvc_nodes_dict[node_id] = {"x": item["xy"][0], "y": item["xy"][1]}

In [17]:
## Reverse-engineering of GPS coordinates
gps_points = np.array(
    [[csvt_nodes_dict[node_id]["lon"], csvt_nodes_dict[node_id]["lat"]] for node_id in csvt_node_ids]).astype(
    np.float64)
xy_points = np.array(
    [[mmvc_nodes_dict[node_id]["x"], mmvc_nodes_dict[node_id]["y"]] for node_id in mmvc_node_ids]).astype(
    np.float64)


# Assumed projection includes rotation, translation and scaling
def affine_transform(params, xy_points):
    a, b, c, d, e, f = params
    x, y = xy_points[:, 0], xy_points[:, 1]
    lon = a * x + b * y + e
    lat = c * x + d * y + f
    return np.vstack([lon, lat]).T


def error_func(params, xy_points, gps_points):
    transformed = affine_transform(params, xy_points)
    return (transformed - gps_points).flatten()


initial_guess = [1, 0, 0, 1, 0, 0]  # identity
result = least_squares(error_func, initial_guess, method="lm", args=(xy_points, gps_points))

# [-4.25994981e-06, -8.74119271e-06, 8.11700876e-06, -3.95042166e-06, 1.14014210e+02, 2.26438070e+01]
opt_params = result.x
print("Optimal parameters: ", opt_params)

transformed_gps_points = affine_transform(opt_params, xy_points)
print("Actual GPS points:")
print(gps_points)
print("Transformed GPS points:")
print(transformed_gps_points)

print("Mean error: ", np.mean(np.sqrt(np.sum(np.power(transformed_gps_points - gps_points, 2), axis=1))))

Optimal parameters:  [-4.25994981e-06 -8.74119271e-06  8.11700876e-06 -3.95042166e-06
  1.14014210e+02  2.26438070e+01]
Actual GPS points:
[[114.040536  22.656135]
 [114.039998  22.645723]
 [114.029911  22.646878]
 [114.03589   22.6473  ]
 [114.030613  22.649589]
 [114.037547  22.599173]
 [114.035699  22.601163]
 [114.005028  22.676499]
 [114.005181  22.679942]
 [114.055657  22.625086]
 [114.050207  22.625298]
 [114.060067  22.615359]]
Transformed GPS points:
[[114.04052899  22.65611959]
 [114.03997597  22.64572208]
 [114.02988292  22.64687848]
 [114.03585077  22.64730729]
 [114.03059047  22.64959076]
 [114.03755338  22.59917522]
 [114.03568483  22.6011462 ]
 [114.0049825   22.67660391]
 [114.00526788  22.67984663]
 [114.05564635  22.62508374]
 [114.05031317  22.62528707]
 [114.06005677  22.61538402]]
Mean error:  4.552703110455817e-05
